In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Cài đặt 
!pip install xgboost
!pip install scikit-learn==1.2.2 imbalanced-learn==0.10.1
import warnings
warnings.simplefilter('ignore', FutureWarning)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.13.0
    Uninstalling imbalanced-learn-0.13.0:
      Successfully uninstalled imbalanced-learn-0.13.0


In [2]:
import pandas as pd

data_filled = pd.read_csv('/kaggle/input/clean-aws-month/final_data/filled_data_april.csv')

# 1. Xử lý dữ liệu bị thiếu

In [3]:
# df_cleaned = data.copy()
# df_cleaned = df_cleaned.drop(columns=['B04B', 'B05B', 'B06B', 'VSB', 'CIN'])

# df_cleaned = df_cleaned[~((df_cleaned['AWS'] == -np.inf) | (df_cleaned['AWS'].isnull()))]
# df_cleaned = df_cleaned[~((df_cleaned['SLHF'] == 9999) | (df_cleaned['SSHF'] == 9999))]

In [4]:
# data_filled = df_cleaned.fillna(df_cleaned.median(numeric_only=True))

# 2. Chia train test

In [5]:
def split_by_day(df):
    df['datetime'] = pd.to_datetime(df['datetime'])
    df['date'] = df['datetime'].dt.date

    # Lấy danh sách ngày duy nhất
    unique_days = sorted(df['date'].unique())
    n_train_days = int(0.8 * len(unique_days))

    # Tách theo ngày
    train_days = unique_days[:n_train_days]
    test_days = unique_days[n_train_days:]

    train_df = df[df['date'].isin(train_days)].copy()
    test_df = df[df['date'].isin(test_days)].copy()

    # In thông tin
    print(f"Số ngày trong train: {len(train_days)}")
    print(f"Số ngày trong test: {len(test_days)}")
    print(f"Train shape: {train_df.shape}")
    print(f"Test shape: {test_df.shape}")
    print(f"Ngày trong train:\n{np.array(train_days)}")
    print(f"Ngày trong test:\n{np.array(test_days)}")

    # Xóa cột phụ trợ 'date'
    train_df.drop(columns='date', inplace=True)
    test_df.drop(columns='date', inplace=True)

    return train_df, test_df

# Chia lại cho từng tháng
train_df, test_df = split_by_day(data_filled)

Số ngày trong train: 48
Số ngày trong test: 12
Train shape: (249931, 34)
Test shape: (55915, 34)
Ngày trong train:
[datetime.date(2019, 4, 1) datetime.date(2019, 4, 2)
 datetime.date(2019, 4, 3) datetime.date(2019, 4, 4)
 datetime.date(2019, 4, 5) datetime.date(2019, 4, 6)
 datetime.date(2019, 4, 7) datetime.date(2019, 4, 8)
 datetime.date(2019, 4, 9) datetime.date(2019, 4, 10)
 datetime.date(2019, 4, 11) datetime.date(2019, 4, 12)
 datetime.date(2019, 4, 13) datetime.date(2019, 4, 14)
 datetime.date(2019, 4, 15) datetime.date(2019, 4, 16)
 datetime.date(2019, 4, 17) datetime.date(2019, 4, 18)
 datetime.date(2019, 4, 19) datetime.date(2019, 4, 20)
 datetime.date(2019, 4, 21) datetime.date(2019, 4, 22)
 datetime.date(2019, 4, 23) datetime.date(2019, 4, 24)
 datetime.date(2019, 4, 25) datetime.date(2019, 4, 26)
 datetime.date(2019, 4, 27) datetime.date(2019, 4, 28)
 datetime.date(2019, 4, 29) datetime.date(2019, 4, 30)
 datetime.date(2020, 4, 1) datetime.date(2020, 4, 2)
 datetime.date(2

In [6]:
X_train = train_df.drop(columns=['AWS', 'datetime', 'row', 'col'])
y_train = (train_df['AWS'] > 0).astype(int)

X_test = test_df.drop(columns=['AWS', 'datetime', 'row', 'col'])
y_test = (test_df['AWS'] > 0).astype(int)

# 3. Chuẩn hóa

In [7]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# 4. Chọn feature

In [8]:
top_features =  ['CAPE', 'KX', 'R500', 'R850', 'TCLW', 'TCW', 'U250', 'U850', 'V250', 'V850', 'B10B', 'B11B', 'B14B', 'I2B', 'I4B', 'IRB', 'WVB']

In [9]:
X_train_selected = X_train_scaled[top_features]
X_test_selected = X_test_scaled[top_features]

In [10]:
#Performance evaluation
def print_scores(y_true, y_pred):
  print(classification_report(y_true, y_pred))

# 5. Thêm class weight

In [11]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(random_state=42, n_estimators = 200)
model_xgb.fit(X_train, y_train)

y_pred = model_xgb.predict(X_test)
print_scores( y_test, y_pred)

              precision    recall  f1-score   support

           0       0.93      0.96      0.94     50316
           1       0.48      0.33      0.39      5599

    accuracy                           0.90     55915
   macro avg       0.71      0.64      0.67     55915
weighted avg       0.88      0.90      0.89     55915



In [12]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(random_state=42, n_estimators = 200)
model_xgb.fit(X_train_scaled, y_train)

y_pred = model_xgb.predict(X_test_scaled)
print_scores( y_test, y_pred)

              precision    recall  f1-score   support

           0       0.93      0.96      0.94     50316
           1       0.48      0.33      0.39      5599

    accuracy                           0.90     55915
   macro avg       0.71      0.64      0.67     55915
weighted avg       0.88      0.90      0.89     55915



In [13]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(random_state=42, n_estimators = 200, scale_pos_weight= 13.09729821196909)
model_xgb.fit(X_train_scaled, y_train)

y_pred = model_xgb.predict(X_test_scaled)
print_scores( y_test, y_pred)

              precision    recall  f1-score   support

           0       0.96      0.83      0.89     50316
           1       0.30      0.65      0.41      5599

    accuracy                           0.81     55915
   macro avg       0.63      0.74      0.65     55915
weighted avg       0.89      0.81      0.84     55915



In [14]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(random_state=42, n_estimators = 200, scale_pos_weight= 13.09729821196909)
model_xgb.fit(X_train_selected, y_train)

y_pred = model_xgb.predict(X_test_selected)
print_scores( y_test, y_pred)

              precision    recall  f1-score   support

           0       0.95      0.80      0.87     50316
           1       0.26      0.64      0.37      5599

    accuracy                           0.79     55915
   macro avg       0.61      0.72      0.62     55915
weighted avg       0.88      0.79      0.82     55915



=> Giải pháp:

1. **Giữ mô hình đã tối ưu + threshold = 0.6** để triển khai hoặc đánh giá cuối cùng.
2. Thử thêm:

   * Tối ưu `max_depth`, `gamma`, `min_child_weight`, `subsample` bằng GridSearchCV hoặc Optuna.
   * Dùng **cross-validation phân tầng (StratifiedKFold)** để đánh giá mô hình ổn định hơn.
3. Nếu cần **giải thích mô hình**, có thể dùng SHAP để giải thích đóng góp các feature.

**Tuning XGBoost không chọn đặc trưng với optuna**

In [15]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=13.09729821196909,
    random_state=42
)
model_xgb.fit(X_train_scaled, y_train)


y_pred = model_xgb.predict(X_test_scaled)
print_scores( y_test, y_pred)

y_probs = model_xgb.predict_proba(X_test_scaled)[:,1]

best_thresh = 0.0
best_f1 = 0.0
for t in np.arange(0.1, 0.9, 0.05):
    y_pred = (y_probs >= t).astype(int)
    f1 = f1_score(y_test, y_pred)
    if f1 > best_f1:
        best_f1 = f1
        best_thresh = t

print(f"Best threshold: {best_thresh}, F1: {best_f1}")


              precision    recall  f1-score   support

           0       0.97      0.76      0.85     50316
           1       0.27      0.80      0.41      5599

    accuracy                           0.76     55915
   macro avg       0.62      0.78      0.63     55915
weighted avg       0.90      0.76      0.81     55915

Best threshold: 0.7000000000000002, F1: 0.4369704417566781


In [16]:
import optuna
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import xgboost as xgb


THRESHOLD = best_thresh
SCALE_POS_WEIGHT =  13.09729821196909 

def objective(trial):
    # Thử các tham số
    param = {
        'n_estimators': 200,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'scale_pos_weight': SCALE_POS_WEIGHT,
        'random_state': 42,
        'verbosity': 0,
        'n_jobs': -1,
        'use_label_encoder': False,
        'eval_metric': 'logloss'
    }

    model = xgb.XGBClassifier(**param)
    model.fit(X_train_scaled, y_train)

    # Dự đoán xác suất
    y_probs = model.predict_proba(X_test_scaled)[:, 1]
    y_pred = (y_probs >= THRESHOLD).astype(int)

    # Trả về F1-score của lớp mưa (label=1)
    return f1_score(y_test, y_pred, pos_label=1)

# Tối ưu với Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # Có thể tăng trials lên 100+

# In kết quả tốt nhất
print("Best trial:")
print(study.best_trial)
print("Best hyperparameters:")
for key, value in study.best_trial.params.items():
    print(f"{key}: {value}")

# Huấn luyện lại với tham số tốt nhất
best_params = study.best_trial.params
best_params.update({
    'n_estimators': 200,
    'scale_pos_weight': SCALE_POS_WEIGHT,
    'random_state': 42,
    'use_label_encoder': False,
    'eval_metric': 'logloss'
})

best_model = xgb.XGBClassifier(**best_params)
best_model.fit(X_train_scaled, y_train)

# Dự đoán với threshold
y_probs = best_model.predict_proba(X_test_scaled)[:, 1]
y_pred_thresh = (y_probs >= THRESHOLD).astype(int)

# In kết quả
from sklearn.metrics import classification_report
print(f"\nClassification report with threshold = {THRESHOLD}:")
print(classification_report(y_test, y_pred_thresh))

[I 2025-05-04 04:22:06,199] A new study created in memory with name: no-name-09833a78-8e7d-462b-9670-bd99153bbc82
[I 2025-05-04 04:22:10,114] Trial 0 finished with value: 0.4335562471032246 and parameters: {'learning_rate': 0.12290323182489198, 'max_depth': 6, 'min_child_weight': 9, 'subsample': 0.8753470916392962, 'colsample_bytree': 0.8829669140688723, 'gamma': 1.7281584461521922}. Best is trial 0 with value: 0.4335562471032246.
[I 2025-05-04 04:22:18,507] Trial 1 finished with value: 0.4096050803730899 and parameters: {'learning_rate': 0.01738879074110561, 'max_depth': 8, 'min_child_weight': 1, 'subsample': 0.7783532427626169, 'colsample_bytree': 0.6743134064739722, 'gamma': 3.2633661223593253}. Best is trial 0 with value: 0.4335562471032246.
[I 2025-05-04 04:22:22,872] Trial 2 finished with value: 0.40415795717756053 and parameters: {'learning_rate': 0.03728966065239204, 'max_depth': 6, 'min_child_weight': 9, 'subsample': 0.7615116625499359, 'colsample_bytree': 0.8817925484303584, 

Best trial:
FrozenTrial(number=45, state=1, values=[0.445458220246001], datetime_start=datetime.datetime(2025, 5, 4, 4, 25, 4, 591282), datetime_complete=datetime.datetime(2025, 5, 4, 4, 25, 7, 520835), params={'learning_rate': 0.20866845677801613, 'max_depth': 4, 'min_child_weight': 6, 'subsample': 0.8331184048355713, 'colsample_bytree': 0.6260333690168361, 'gamma': 0.3571860177779096}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.3, log=False, low=0.01, step=None), 'max_depth': IntDistribution(high=10, log=False, low=3, step=1), 'min_child_weight': IntDistribution(high=10, log=False, low=1, step=1), 'subsample': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'colsample_bytree': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'gamma': FloatDistribution(high=5.0, log=False, low=0.0, step=None)}, trial_id=45, value=None)
Best hyperparameters:
learning_rate: 0.20866845677801613
max_depth: 4
min_c

**Tuning XGBoost có chọn đặc trưng với optuna**

In [17]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight= 13.09729821196909,
    random_state=42
)
model_xgb.fit(X_train_selected, y_train)


y_pred = model_xgb.predict(X_test_selected)
print_scores( y_test, y_pred)

y_probs = model_xgb.predict_proba(X_test_selected)[:,1]

best_thresh = 0.0
best_f1 = 0.0
for t in np.arange(0.1, 0.9, 0.05):
    y_pred = (y_probs >= t).astype(int)
    f1 = f1_score(y_test, y_pred)
    if f1 > best_f1:
        best_f1 = f1
        best_thresh = t

print(f"Best threshold: {best_thresh}, F1: {best_f1}")

              precision    recall  f1-score   support

           0       0.97      0.72      0.83     50316
           1       0.25      0.83      0.38      5599

    accuracy                           0.73     55915
   macro avg       0.61      0.78      0.61     55915
weighted avg       0.90      0.73      0.78     55915

Best threshold: 0.7000000000000002, F1: 0.3975746585821954


In [18]:
import optuna
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import xgboost as xgb


THRESHOLD = best_thresh
SCALE_POS_WEIGHT =  13.09729821196909 

def objective(trial):
    # Thử các tham số
    param = {
        'n_estimators': 200,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'scale_pos_weight': SCALE_POS_WEIGHT,
        'random_state': 42,
        'verbosity': 0,
        'n_jobs': -1,
        'use_label_encoder': False,
        'eval_metric': 'logloss'
    }

    model = xgb.XGBClassifier(**param)
    model.fit(X_train_selected, y_train)

    # Dự đoán xác suất
    y_probs = model.predict_proba(X_test_selected)[:, 1]
    y_pred = (y_probs >= THRESHOLD).astype(int)

    # Trả về F1-score của lớp mưa (label=1)
    return f1_score(y_test, y_pred, pos_label=1)

# Tối ưu với Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # Có thể tăng trials lên 100+

# In kết quả tốt nhất
print("Best trial:")
print(study.best_trial)
print("Best hyperparameters:")
for key, value in study.best_trial.params.items():
    print(f"{key}: {value}")

# Huấn luyện lại với tham số tốt nhất
best_params = study.best_trial.params
best_params.update({
    'n_estimators': 200,
    'scale_pos_weight': SCALE_POS_WEIGHT,
    'random_state': 42,
    'use_label_encoder': False,
    'eval_metric': 'logloss'
})

best_model = xgb.XGBClassifier(**best_params)
best_model.fit(X_train_scaled, y_train)

# Dự đoán với threshold
y_probs = best_model.predict_proba(X_test_scaled)[:, 1]
y_pred_thresh = (y_probs >= THRESHOLD).astype(int)

# In kết quả
from sklearn.metrics import classification_report
# print("\nClassification report with threshold = 0.75:")
print(f"\nClassification report with threshold = {THRESHOLD}:")
print(classification_report(y_test, y_pred_thresh))

[I 2025-05-04 04:25:25,144] A new study created in memory with name: no-name-5f726245-4997-4b6c-8cf9-85e40d121b4b
[I 2025-05-04 04:25:31,370] Trial 0 finished with value: 0.3921849280755743 and parameters: {'learning_rate': 0.10529495853619515, 'max_depth': 9, 'min_child_weight': 9, 'subsample': 0.5483078595211718, 'colsample_bytree': 0.8550417500030691, 'gamma': 2.630475152259553}. Best is trial 0 with value: 0.3921849280755743.
[I 2025-05-04 04:25:35,203] Trial 1 finished with value: 0.3705591843237216 and parameters: {'learning_rate': 0.2337089649998992, 'max_depth': 8, 'min_child_weight': 5, 'subsample': 0.6275330815358686, 'colsample_bytree': 0.700651440346747, 'gamma': 0.6985544313760644}. Best is trial 0 with value: 0.3921849280755743.
[I 2025-05-04 04:25:38,781] Trial 2 finished with value: 0.3962366172812804 and parameters: {'learning_rate': 0.01931033594077459, 'max_depth': 7, 'min_child_weight': 2, 'subsample': 0.8704129772200201, 'colsample_bytree': 0.5174361413131987, 'gam

Best trial:
FrozenTrial(number=6, state=1, values=[0.430336696519669], datetime_start=datetime.datetime(2025, 5, 4, 4, 25, 46, 240988), datetime_complete=datetime.datetime(2025, 5, 4, 4, 25, 48, 412852), params={'learning_rate': 0.2259373587023201, 'max_depth': 4, 'min_child_weight': 2, 'subsample': 0.6746386792178822, 'colsample_bytree': 0.7629284117838611, 'gamma': 1.5786263739636737}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.3, log=False, low=0.01, step=None), 'max_depth': IntDistribution(high=10, log=False, low=3, step=1), 'min_child_weight': IntDistribution(high=10, log=False, low=1, step=1), 'subsample': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'colsample_bytree': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'gamma': FloatDistribution(high=5.0, log=False, low=0.0, step=None)}, trial_id=6, value=None)
Best hyperparameters:
learning_rate: 0.2259373587023201
max_depth: 4
min_chi

**Đánh giá:**

Lớp mưa:
* Recall = 0.80: Mô hình phát hiện được 80% các trường hợp có mưa → rất tốt trong bối cảnh nhãn lệch (khó học).
* F1-score = 0.70: Mức F1-score cao hơn mô hình mặc định (~0.57) hoặc các cấu hình thủ công trước đó.
* Precision = 0.61: Có một số false positives (dự đoán sai là mưa)

Lớp không mưa: Rất tốt, vẫn giữ precision 0.93 và F1-score 0.89

**Không bắt buộc phải cân bằng dữ liệu khi dùng XGBoost vì đã dùng scale_pos_weight, threshold và tham số bằng optuna) nhưng vẫn nên thử**

=> Chọn đặc trưng hay không thì kết quả cũng tương tự nhau.

# 6. Cân bằng dữ liệu (Oversampling)

## 6.1 SMOTE

In [19]:
# SMOTE
from imblearn.over_sampling import SMOTE

sm = SMOTE()

X_sm, y_sm = sm.fit_resample(X_train, y_train)
X_sm.shape, y_sm.shape

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_sm)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

X_train_selected = X_train_scaled[top_features]
X_test_selected = X_test_scaled[top_features]

In [20]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(random_state=42, n_estimators = 200)
model_xgb.fit(X_sm, y_sm)

y_pred = model_xgb.predict(X_test)
print_scores( y_test, y_pred)

              precision    recall  f1-score   support

           0       0.94      0.93      0.93     50316
           1       0.41      0.44      0.42      5599

    accuracy                           0.88     55915
   macro avg       0.67      0.68      0.68     55915
weighted avg       0.88      0.88      0.88     55915



In [21]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(random_state=42, n_estimators = 200)
model_xgb.fit(X_train_scaled, y_sm)

y_pred = model_xgb.predict(X_test_scaled)
print_scores( y_test, y_pred)

              precision    recall  f1-score   support

           0       0.94      0.93      0.93     50316
           1       0.42      0.49      0.45      5599

    accuracy                           0.88     55915
   macro avg       0.68      0.71      0.69     55915
weighted avg       0.89      0.88      0.89     55915



In [22]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(random_state=42, n_estimators = 200)
model_xgb.fit(X_train_scaled[top_features], y_sm)

y_pred = model_xgb.predict(X_test_selected)
print_scores( y_test, y_pred)

              precision    recall  f1-score   support

           0       0.94      0.83      0.88     50316
           1       0.27      0.56      0.36      5599

    accuracy                           0.80     55915
   macro avg       0.61      0.70      0.62     55915
weighted avg       0.88      0.80      0.83     55915



In [23]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(random_state=42, n_estimators = 200, min_child_weight=5)
model_xgb.fit(X_train_scaled, y_sm)

y_pred = model_xgb.predict(X_test_scaled)
print_scores( y_test, y_pred)

              precision    recall  f1-score   support

           0       0.94      0.92      0.93     50316
           1       0.37      0.43      0.40      5599

    accuracy                           0.87     55915
   macro avg       0.65      0.68      0.66     55915
weighted avg       0.88      0.87      0.87     55915



=> Thêm min_child_weight có cải thiện

In [24]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(random_state=42, n_estimators = 200, scale_pos_weight= 13.09729821196909)
model_xgb.fit(X_train_scaled, y_sm)

y_pred = model_xgb.predict(X_test_scaled)
print_scores( y_test, y_pred)

              precision    recall  f1-score   support

           0       0.96      0.74      0.83     50316
           1       0.23      0.72      0.35      5599

    accuracy                           0.73     55915
   macro avg       0.60      0.73      0.59     55915
weighted avg       0.89      0.73      0.78     55915



=> Thêm scale_pos_weight giảm

In [25]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    # scale_pos_weight= 13.09729821196909,
    random_state=42,
    min_child_weight=5
    
)

model_xgb.fit(X_train_scaled, y_sm)

y_pred = model_xgb.predict(X_test_scaled)
print_scores(y_test, y_pred)

y_probs = model_xgb.predict_proba(X_test_scaled)[:,1]

best_thresh = 0.0
best_f1 = 0.0
for t in np.arange(0.1, 0.9, 0.05):
    y_pred = (y_probs >= t).astype(int)
    f1 = f1_score(y_test, y_pred)
    if f1 > best_f1:
        best_f1 = f1
        best_thresh = t

print(f"Best threshold: {best_thresh}, F1: {best_f1}")

              precision    recall  f1-score   support

           0       0.95      0.87      0.91     50316
           1       0.35      0.62      0.45      5599

    accuracy                           0.85     55915
   macro avg       0.65      0.75      0.68     55915
weighted avg       0.89      0.85      0.87     55915

Best threshold: 0.5000000000000001, F1: 0.4486746361746362


In [26]:
import optuna
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import xgboost as xgb

THRESHOLD = best_thresh
# SCALE_POS_WEIGHT =  13.09729821196909 

def objective(trial):
    # Thử các tham số
    param = {
        'n_estimators': 200,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        # 'scale_pos_weight': SCALE_POS_WEIGHT,
        'random_state': 42,
        'verbosity': 0,
        'n_jobs': -1,
        'use_label_encoder': False,
        'eval_metric': 'logloss'
    }

    model = xgb.XGBClassifier(**param)
    model.fit(X_train_scaled, y_sm)

    # Dự đoán xác suất
    y_probs = model.predict_proba(X_test_scaled)[:, 1]
    y_pred = (y_probs >= THRESHOLD).astype(int)

    # Trả về F1-score của lớp mưa (label=1)
    return f1_score(y_test, y_pred, pos_label=1)


# Tối ưu với Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)  # Có thể tăng trials lên 100+

# In kết quả tốt nhất
print("Best trial:")
print(study.best_trial)
print("Best hyperparameters:")
for key, value in study.best_trial.params.items():
    print(f"{key}: {value}")

# Huấn luyện lại với tham số tốt nhất
best_params = study.best_trial.params
best_params.update({
    'n_estimators': 200,
    # 'scale_pos_weight': SCALE_POS_WEIGHT,
    'random_state': 42,
    'use_label_encoder': False,
    'eval_metric': 'logloss'
})

best_model = xgb.XGBClassifier(**best_params)
best_model.fit(X_train_scaled, y_sm)

# Dự đoán với threshold
y_probs = best_model.predict_proba(X_test_scaled)[:, 1]
y_pred_thresh = (y_probs >= THRESHOLD).astype(int)

# In kết quả
from sklearn.metrics import classification_report
print(f"\nClassification report with threshold = {THRESHOLD}:")
print(classification_report(y_test, y_pred_thresh))

[I 2025-05-04 04:28:34,155] A new study created in memory with name: no-name-61c5ca0c-08e0-470c-900a-56ba01e5f783
[I 2025-05-04 04:28:42,722] Trial 0 finished with value: 0.4516320474777448 and parameters: {'learning_rate': 0.16081083938860552, 'max_depth': 6, 'min_child_weight': 1, 'subsample': 0.6680309270242579, 'colsample_bytree': 0.7842062869304804, 'gamma': 3.941145457074967}. Best is trial 0 with value: 0.4516320474777448.
[I 2025-05-04 04:28:48,158] Trial 1 finished with value: 0.43796610169491523 and parameters: {'learning_rate': 0.12600854808702144, 'max_depth': 4, 'min_child_weight': 10, 'subsample': 0.5545070920087263, 'colsample_bytree': 0.6598016692251338, 'gamma': 3.230115049796632}. Best is trial 0 with value: 0.4516320474777448.
[I 2025-05-04 04:28:54,208] Trial 2 finished with value: 0.3717773733697301 and parameters: {'learning_rate': 0.018778733958757832, 'max_depth': 4, 'min_child_weight': 8, 'subsample': 0.54767876518066, 'colsample_bytree': 0.8528077076930538, 'g

Best trial:
FrozenTrial(number=0, state=1, values=[0.4516320474777448], datetime_start=datetime.datetime(2025, 5, 4, 4, 28, 34, 157173), datetime_complete=datetime.datetime(2025, 5, 4, 4, 28, 42, 722069), params={'learning_rate': 0.16081083938860552, 'max_depth': 6, 'min_child_weight': 1, 'subsample': 0.6680309270242579, 'colsample_bytree': 0.7842062869304804, 'gamma': 3.941145457074967}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.3, log=False, low=0.01, step=None), 'max_depth': IntDistribution(high=10, log=False, low=3, step=1), 'min_child_weight': IntDistribution(high=10, log=False, low=1, step=1), 'subsample': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'colsample_bytree': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'gamma': FloatDistribution(high=5.0, log=False, low=0.0, step=None)}, trial_id=0, value=None)
Best hyperparameters:
learning_rate: 0.16081083938860552
max_depth: 6
min_c

In [27]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    # scale_pos_weight= 13.09729821196909,
    random_state=42,
    min_child_weight=5
    
)

model_xgb.fit(X_train_scaled[top_features], y_sm)

y_pred = model_xgb.predict(X_test_selected)
print_scores(y_test, y_pred)

y_probs = model_xgb.predict_proba(X_test_selected)[:,1]

best_thresh = 0.0
best_f1 = 0.0
for t in np.arange(0.1, 0.9, 0.05):
    y_pred = (y_probs >= t).astype(int)
    f1 = f1_score(y_test, y_pred)
    if f1 > best_f1:
        best_f1 = f1
        best_thresh = t

print(f"Best threshold: {best_thresh}, F1: {best_f1}")

              precision    recall  f1-score   support

           0       0.96      0.81      0.87     50316
           1       0.28      0.66      0.39      5599

    accuracy                           0.79     55915
   macro avg       0.62      0.74      0.63     55915
weighted avg       0.89      0.79      0.83     55915

Best threshold: 0.6500000000000001, F1: 0.3995312607706624


In [28]:
import optuna
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import xgboost as xgb


THRESHOLD = best_thresh
# SCALE_POS_WEIGHT =  13.09729821196909 

def objective(trial):
    # Thử các tham số
    param = {
        'n_estimators': 200,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        # 'scale_pos_weight': SCALE_POS_WEIGHT,
        'random_state': 42,
        'verbosity': 0,
        'n_jobs': -1,
        'use_label_encoder': False,
        'eval_metric': 'logloss'
    }

    model = xgb.XGBClassifier(**param)
    model.fit(X_train_scaled[top_features], y_sm)

    # Dự đoán xác suất
    y_probs = model.predict_proba(X_test_selected)[:, 1]
    y_pred = (y_probs >= THRESHOLD).astype(int)

    # Trả về F1-score của lớp mưa (label=1)
    return f1_score(y_test, y_pred, pos_label=1)


# Tối ưu với Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)  # Có thể tăng trials lên 100+

# In kết quả tốt nhất
print("Best trial:")
print(study.best_trial)
print("Best hyperparameters:")
for key, value in study.best_trial.params.items():
    print(f"{key}: {value}")

# Huấn luyện lại với tham số tốt nhất
best_params = study.best_trial.params
best_params.update({
    'n_estimators': 200,
    # 'scale_pos_weight': SCALE_POS_WEIGHT,
    'random_state': 42,
    'use_label_encoder': False,
    'eval_metric': 'logloss'
})

best_model = xgb.XGBClassifier(**best_params)
best_model.fit(X_train_scaled[top_features], y_sm)

# Dự đoán với threshold
y_probs = best_model.predict_proba(X_test_selected)[:, 1]
y_pred_thresh = (y_probs >= THRESHOLD).astype(int)

# In kết quả
from sklearn.metrics import classification_report
print(f"\nClassification report with threshold = {THRESHOLD}:")
print(classification_report(y_test, y_pred_thresh))

[I 2025-05-04 04:32:35,466] A new study created in memory with name: no-name-c28b86c3-a835-4ea5-b7de-2198bae4de31
[I 2025-05-04 04:32:41,349] Trial 0 finished with value: 0.39097523333578305 and parameters: {'learning_rate': 0.11596671917265597, 'max_depth': 7, 'min_child_weight': 4, 'subsample': 0.697395400684699, 'colsample_bytree': 0.7201606045635833, 'gamma': 2.4257111650123147}. Best is trial 0 with value: 0.39097523333578305.
[I 2025-05-04 04:32:49,676] Trial 1 finished with value: 0.3886628508378733 and parameters: {'learning_rate': 0.060514995150434, 'max_depth': 9, 'min_child_weight': 1, 'subsample': 0.6007186706121943, 'colsample_bytree': 0.8119609942475609, 'gamma': 3.3899721198518717}. Best is trial 0 with value: 0.39097523333578305.
[I 2025-05-04 04:32:57,459] Trial 2 finished with value: 0.38477988348941816 and parameters: {'learning_rate': 0.18961712750571172, 'max_depth': 8, 'min_child_weight': 9, 'subsample': 0.5530902497382189, 'colsample_bytree': 0.7020605657730364, 

Best trial:
FrozenTrial(number=10, state=1, values=[0.42155895411938826], datetime_start=datetime.datetime(2025, 5, 4, 4, 33, 37, 588185), datetime_complete=datetime.datetime(2025, 5, 4, 4, 33, 41, 260378), params={'learning_rate': 0.2989291547723118, 'max_depth': 3, 'min_child_weight': 7, 'subsample': 0.7902546060840123, 'colsample_bytree': 0.5188413789142464, 'gamma': 1.4933284547580334}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.3, log=False, low=0.01, step=None), 'max_depth': IntDistribution(high=10, log=False, low=3, step=1), 'min_child_weight': IntDistribution(high=10, log=False, low=1, step=1), 'subsample': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'colsample_bytree': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'gamma': FloatDistribution(high=5.0, log=False, low=0.0, step=None)}, trial_id=10, value=None)
Best hyperparameters:
learning_rate: 0.2989291547723118
max_depth: 3
min

**Đối với dùng SMOTE:** chỉ scale cao hơn scale xong chọn đặc trưng nhưng không SMOTE cao hơn đã SMOTE.

## 6.2 SMOTE ENN

In [29]:
# SMOTE ENN
from imblearn.combine import SMOTEENN

smenn = SMOTEENN()

X_smenn, y_smenn = smenn.fit_resample(X_train, y_train)
X_smenn.shape, y_sm.shape

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_smenn)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

X_train_selected = X_train_scaled[top_features]
X_test_selected = X_test_scaled[top_features]

In [30]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(random_state=42, n_estimators = 200)
model_xgb.fit(X_smenn, y_smenn)

y_pred = model_xgb.predict(X_test)
print_scores(y_test, y_pred)

              precision    recall  f1-score   support

           0       0.95      0.89      0.92     50316
           1       0.35      0.55      0.43      5599

    accuracy                           0.85     55915
   macro avg       0.65      0.72      0.67     55915
weighted avg       0.89      0.85      0.87     55915



In [31]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(random_state=42, n_estimators = 200)
model_xgb.fit(X_train_scaled, y_smenn)

y_pred = model_xgb.predict(X_test_scaled)
print_scores(y_test, y_pred)

              precision    recall  f1-score   support

           0       0.95      0.89      0.92     50316
           1       0.35      0.54      0.43      5599

    accuracy                           0.85     55915
   macro avg       0.65      0.72      0.67     55915
weighted avg       0.89      0.85      0.87     55915



In [32]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(random_state=42, n_estimators = 200)
model_xgb.fit(X_train_scaled[top_features], y_smenn)

y_pred = model_xgb.predict(X_test_selected)
print_scores(y_test, y_pred)

              precision    recall  f1-score   support

           0       0.95      0.82      0.88     50316
           1       0.26      0.59      0.36      5599

    accuracy                           0.79     55915
   macro avg       0.61      0.70      0.62     55915
weighted avg       0.88      0.79      0.83     55915



=> Không chọn đặc trưng, scale hay không cũng như nhau.

In [33]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(random_state=42, n_estimators = 200, min_child_weight=5)
model_xgb.fit(X_train_scaled, y_smenn)

y_pred = model_xgb.predict(X_test_scaled)
print_scores(y_test, y_pred)

              precision    recall  f1-score   support

           0       0.95      0.89      0.92     50316
           1       0.35      0.55      0.43      5599

    accuracy                           0.85     55915
   macro avg       0.65      0.72      0.67     55915
weighted avg       0.89      0.85      0.87     55915



In [34]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    # scale_pos_weight= 13.09729821196909,
    random_state=42
)
model_xgb.fit(X_train_scaled, y_smenn)

y_pred = model_xgb.predict(X_test_scaled)
print_scores( y_test, y_pred)

y_probs = model_xgb.predict_proba(X_test_scaled)[:,1]

best_thresh = 0.0
best_f1 = 0.0
for t in np.arange(0.1, 0.9, 0.05):
    y_pred = (y_probs >= t).astype(int)
    f1 = f1_score(y_test, y_pred)
    if f1 > best_f1:
        best_f1 = f1
        best_thresh = t

print(f"Best threshold: {best_thresh}, F1: {best_f1}")

              precision    recall  f1-score   support

           0       0.95      0.85      0.90     50316
           1       0.33      0.64      0.43      5599

    accuracy                           0.83     55915
   macro avg       0.64      0.75      0.67     55915
weighted avg       0.89      0.83      0.85     55915

Best threshold: 0.6000000000000002, F1: 0.43656821378340366


In [35]:
import optuna
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import xgboost as xgb

THRESHOLD = best_thresh
# SCALE_POS_WEIGHT =  13.09729821196909

def objective(trial):
    # Thử các tham số
    param = {
        'n_estimators': 200,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        # 'scale_pos_weight': SCALE_POS_WEIGHT,
        'random_state': 42,
        'verbosity': 0,
        'n_jobs': -1,
        'use_label_encoder': False,
        'eval_metric': 'logloss'
    }

    model = xgb.XGBClassifier(**param)
    model.fit(X_train_scaled, y_smenn)

    # Dự đoán xác suất
    y_probs = model.predict_proba(X_test_scaled)[:, 1]
    y_pred = (y_probs >= THRESHOLD).astype(int)

    # Trả về F1-score của lớp mưa (label=1)
    return f1_score(y_test, y_pred, pos_label=1)


# Tối ưu với Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)  # Có thể tăng trials lên 100+

# In kết quả tốt nhất
print("Best trial:")
print(study.best_trial)
print("Best hyperparameters:")
for key, value in study.best_trial.params.items():
    print(f"{key}: {value}")

# Huấn luyện lại với tham số tốt nhất
best_params = study.best_trial.params
best_params.update({
    'n_estimators': 200,
    # 'scale_pos_weight': SCALE_POS_WEIGHT,
    'random_state': 42,
    'use_label_encoder': False,
    'eval_metric': 'logloss'
})

best_model = xgb.XGBClassifier(**best_params)
best_model.fit(X_train_scaled, y_smenn)

# Dự đoán với threshold 
y_probs = best_model.predict_proba(X_test_scaled)[:, 1]
y_pred_thresh = (y_probs >= THRESHOLD).astype(int)

# In kết quả
from sklearn.metrics import classification_report
print(f"\nClassification report with threshold = {THRESHOLD}:")
print(classification_report(y_test, y_pred_thresh))

[I 2025-05-04 04:44:03,443] A new study created in memory with name: no-name-52c34318-8b34-4b68-adcf-8ff52f2a7fe9
[I 2025-05-04 04:44:09,163] Trial 0 finished with value: 0.4381281407035175 and parameters: {'learning_rate': 0.2521620055437126, 'max_depth': 7, 'min_child_weight': 10, 'subsample': 0.9877871626097963, 'colsample_bytree': 0.843030384066573, 'gamma': 3.567652042677088}. Best is trial 0 with value: 0.4381281407035175.
[I 2025-05-04 04:44:16,771] Trial 1 finished with value: 0.4366683095120749 and parameters: {'learning_rate': 0.22730047495459002, 'max_depth': 6, 'min_child_weight': 2, 'subsample': 0.5015973770725426, 'colsample_bytree': 0.8955423564159255, 'gamma': 1.478417163196792}. Best is trial 0 with value: 0.4381281407035175.
[I 2025-05-04 04:44:23,423] Trial 2 finished with value: 0.4369499297389246 and parameters: {'learning_rate': 0.07455627661648312, 'max_depth': 7, 'min_child_weight': 7, 'subsample': 0.7176439905253271, 'colsample_bytree': 0.7208809632851667, 'gam

Best trial:
FrozenTrial(number=18, state=1, values=[0.4537444933920705], datetime_start=datetime.datetime(2025, 5, 4, 4, 45, 58, 390934), datetime_complete=datetime.datetime(2025, 5, 4, 4, 46, 3, 633052), params={'learning_rate': 0.19553694167700647, 'max_depth': 5, 'min_child_weight': 4, 'subsample': 0.8371697143469166, 'colsample_bytree': 0.55995013213359, 'gamma': 3.6813928492860253}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.3, log=False, low=0.01, step=None), 'max_depth': IntDistribution(high=10, log=False, low=3, step=1), 'min_child_weight': IntDistribution(high=10, log=False, low=1, step=1), 'subsample': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'colsample_bytree': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'gamma': FloatDistribution(high=5.0, log=False, low=0.0, step=None)}, trial_id=18, value=None)
Best hyperparameters:
learning_rate: 0.19553694167700647
max_depth: 5
min_c

In [36]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    # scale_pos_weight= 13.09729821196909,
    random_state=42
)
model_xgb.fit(X_train_scaled[top_features], y_smenn)


y_pred = model_xgb.predict(X_test_selected)
print_scores(y_test, y_pred)

y_probs = model_xgb.predict_proba(X_test_selected)[:,1]

best_thresh = 0.0
best_f1 = 0.0
for t in np.arange(0.1, 0.9, 0.05):
    y_pred = (y_probs >= t).astype(int)
    f1 = f1_score(y_test, y_pred)
    if f1 > best_f1:
        best_f1 = f1
        best_thresh = t

print(f"Best threshold: {best_thresh}, F1: {best_f1}")

              precision    recall  f1-score   support

           0       0.96      0.79      0.86     50316
           1       0.26      0.68      0.38      5599

    accuracy                           0.78     55915
   macro avg       0.61      0.73      0.62     55915
weighted avg       0.89      0.78      0.82     55915

Best threshold: 0.6500000000000001, F1: 0.38606891441274543


In [37]:
import optuna
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import xgboost as xgb

THRESHOLD = best_thresh
# SCALE_POS_WEIGHT =  13.09729821196909

def objective(trial):
    # Thử các tham số
    param = {
        'n_estimators': 200,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        # 'scale_pos_weight': SCALE_POS_WEIGHT,
        'random_state': 42,
        'verbosity': 0,
        'n_jobs': -1,
        'use_label_encoder': False,
        'eval_metric': 'logloss'
    }

    model = xgb.XGBClassifier(**param)
    model.fit(X_train_scaled[top_features], y_smenn)

    # Dự đoán xác suất
    y_probs = model.predict_proba(X_test_selected)[:, 1]
    y_pred = (y_probs >= THRESHOLD).astype(int)

    # Trả về F1-score của lớp mưa (label=1)
    return f1_score(y_test, y_pred, pos_label=1)


# Tối ưu với Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)  # Có thể tăng trials lên 100+

# In kết quả tốt nhất
print("Best trial:")
print(study.best_trial)
print("Best hyperparameters:")
for key, value in study.best_trial.params.items():
    print(f"{key}: {value}")

# Huấn luyện lại với tham số tốt nhất
best_params = study.best_trial.params
best_params.update({
    'n_estimators': 200,
    # 'scale_pos_weight': SCALE_POS_WEIGHT,
    'random_state': 42,
    'use_label_encoder': False,
    'eval_metric': 'logloss'
})

best_model = xgb.XGBClassifier(**best_params)
best_model.fit(X_train_scaled[top_features], y_smenn)

# Dự đoán với threshold
y_probs = best_model.predict_proba(X_test_selected)[:, 1]
y_pred_thresh = (y_probs >= THRESHOLD).astype(int)

# In kết quả
from sklearn.metrics import classification_report
print(f"\nClassification report with threshold = {THRESHOLD}:")
print(classification_report(y_test, y_pred_thresh))

[I 2025-05-04 04:47:10,526] A new study created in memory with name: no-name-2e948ad6-d413-42d3-b39f-12cbbae5c208
[I 2025-05-04 04:47:16,751] Trial 0 finished with value: 0.3796019584154638 and parameters: {'learning_rate': 0.07387577981512576, 'max_depth': 9, 'min_child_weight': 6, 'subsample': 0.7751170175214386, 'colsample_bytree': 0.5583121590840239, 'gamma': 2.673301380603133}. Best is trial 0 with value: 0.3796019584154638.
[I 2025-05-04 04:47:20,622] Trial 1 finished with value: 0.4074860900354072 and parameters: {'learning_rate': 0.056338105859110386, 'max_depth': 5, 'min_child_weight': 10, 'subsample': 0.9412970846612885, 'colsample_bytree': 0.9324561828511961, 'gamma': 1.9114516924253784}. Best is trial 1 with value: 0.4074860900354072.
[I 2025-05-04 04:47:23,900] Trial 2 finished with value: 0.3987488415199259 and parameters: {'learning_rate': 0.2994362917936699, 'max_depth': 4, 'min_child_weight': 3, 'subsample': 0.8261141784552808, 'colsample_bytree': 0.6396730679388528, '

Best trial:
FrozenTrial(number=26, state=1, values=[0.4121775325492474], datetime_start=datetime.datetime(2025, 5, 4, 4, 49, 8, 601230), datetime_complete=datetime.datetime(2025, 5, 4, 4, 49, 12, 823406), params={'learning_rate': 0.03412969623229642, 'max_depth': 5, 'min_child_weight': 10, 'subsample': 0.5587134418128034, 'colsample_bytree': 0.8024427679372479, 'gamma': 3.576190704496534}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.3, log=False, low=0.01, step=None), 'max_depth': IntDistribution(high=10, log=False, low=3, step=1), 'min_child_weight': IntDistribution(high=10, log=False, low=1, step=1), 'subsample': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'colsample_bytree': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'gamma': FloatDistribution(high=5.0, log=False, low=0.0, step=None)}, trial_id=26, value=None)
Best hyperparameters:
learning_rate: 0.03412969623229642
max_depth: 5
min

# 7. Kết luận

Mô hình XGBoost ban đầu (không SMOTE hay SMOTE ENN) là tốt nhất.